In [109]:
#Author: Habib Bashour (Greiff lab) ## modified Oct 2024 Gemma Gordon 
# convert from R to python
# https://www.rdocumentation.org/packages/Peptides/versions/2.4.6 
# https://github.com/althonos/peptides.py/tree/main/docs
# https://peptides.readthedocs.io/en/stable/api/peptide.html 

#######PART 1#########

In [2]:
import peptides
import pandas as pd
import numpy as np

In [111]:
#first load/read a dataframe that contains the sequences you are interested in
#this data should include aaSeqAbChain column that contains the sequence of the variable region (fv) of the antibody.

#df = pd.read_csv('/data/localhost/gordon/TNP_Project/datasets/greiff_inputs/greiff_all_datasets.csv')
# rename cols
#df.columns = ['SeqID','Seq','Dataset']

In [ ]:
dataset_name = 'VHH_TSD'

In [ ]:
with open('vhh_tsd.fasta', 'r') as vh_seqs:
    entries = [v.strip('\n') for v in vh_seqs.readlines()]
    headers = [e for e in entries if '>' in e]
    seqs = [e for e in entries if '>' not in e]

df = pd.DataFrame()
df['SeqID'] = [h.strip('>') for h in headers]
df['Seq'] = seqs
df['Dataset'] = dataset_name


In [57]:
#molecular weight etc ------
def get_mw(seq):
  seq = peptides.Peptide(seq) # have to instantiate class first
  mw = seq.molecular_weight()
  return mw

def get_seqlength(seq):
  length = len(seq)
  return length

def get_avresweight(seq):
  seq = peptides.Peptide(seq)
  avresweight = get_mw(seq) / (len(seq))
  return avresweight

In [58]:
df['Molecular Weight'] = df['Seq'].apply(get_mw)
df['Seq Length'] = df['Seq'].apply(get_seqlength)
df['Average Residue Weight'] = df['Seq'].apply(get_avresweight)

In [ ]:
#seq = peptides.Peptide('') # input string sequence

In [60]:
#seq.charge(pH=14, pKscale= 'Lehninger')

In [61]:
#charge----

def get_all_charges(seq):

    all_charges = dict()
    charge_intervals = np.arange(1,15,1) # for pH 1-14
    seq = peptides.Peptide(seq)

    for i in charge_intervals:
        charge = seq.charge(pH=i, pKscale= 'Lehninger')
        all_charges[i] = charge

    return all_charges

In [62]:
#   developability_data = developability_data %>%
#     mutate(!!paste0("AbChain_",charge_intervals[i],"_charge") := unlist(clusterMap(cl, get_charge, seq_input = developability_data$aaSeqAbChain, pH_input = pH_input)))
# }

df['Charges (all pH values)'] = df['Seq'].apply(get_all_charges)
# split into new columns for individual pH values


In [64]:
#pI----

def get_pI(seq):

    seq = peptides.Peptide(seq)
    pI = seq.isoelectric_point(pKscale = "Lehninger")
    return pI 

In [65]:
df['pI'] = df['Seq'].apply(get_pI)

In [67]:
#photochemical----

def get_mol_extcoef(seq):
  
  if seq.count('C') % 2 == 0: 
    return (seq.count('Y')*1490) + (seq.count('W')*5500) + ((seq.count('C')/2)*125)

  else:
    return (seq.count('Y')*1490) + (seq.count('W')*5500) + (((seq.count('C')-1)/2)*125)

def get_mol_extcoef_cysteine_bridges(seq): 

  if seq.count('C') % 2 == 0:
    return (seq.count('C')/2)*125
  
  else:
    return ((seq.count('C')-1)/2)*125

In [68]:
df['Molecular Extinction Coefficient'] = df['Seq'].apply(get_mol_extcoef)
df['Molecular Extinction Coefficient (Cysteine bridges)'] = df['Seq'].apply(get_mol_extcoef_cysteine_bridges)

In [69]:
def get_percent_mol_extcoef(seq):
  
    if seq.count('C') % 2 == 0: 
        mol_extcoef = (seq.count('Y')*1490) + (seq.count('W')*5500) + ((seq.count('C')/2)*125)

    else:
        mol_extcoef = (seq.count('Y')*1490) + (seq.count('W')*5500) + (((seq.count('C')-1)/2)*125)
    
    return (mol_extcoef * 10) / get_mw(seq)

def get_percent_mol_extcoef_cysteine_bridges(seq): 

    if seq.count('C') % 2 == 0:
        mol_extcoef_cys = (seq.count('C')/2)*125

    else:
        mol_extcoef_cys = ((seq.count('C')-1)/2)*125

    return (mol_extcoef_cys * 10) / get_mw(seq)

In [70]:
df['% Molecular Extinction Coefficient'] = df['Seq'].apply(get_percent_mol_extcoef)
df['% Molecular Extinction Coefficient (Cysteine bridges)'] = df['Seq'].apply(get_percent_mol_extcoef_cysteine_bridges)

In [72]:
#indexes---- 
def get_instaindex(seq):
  if len(seq)<3:
   return 'NA'
  else: 
   seq = peptides.Peptide(seq)
   return seq.instability_index()

def get_aliphindex(seq):
   seq = peptides.Peptide(seq)
   return seq.aliphatic_index()

In [73]:
df['Instability Index'] = df['Seq'].apply(get_instaindex)
df['Aliphatic Index'] = df['Seq'].apply(get_aliphindex)

In [74]:
#hydrophobicity and hmom----

def get_hydrophobicity(seq):
  seq = peptides.Peptide(seq)
  return seq.hydrophobicity(scale='Eisenberg')

def get_hmom(seq): 
  seq = peptides.Peptide(seq)
  return seq.hydrophobic_moment(angle=160,window=10)

In [75]:
df['Hydrophobicity'] = df['Seq'].apply(get_hydrophobicity)
df['Hydrophobic moment'] = df['Seq'].apply(get_hmom)

In [76]:

#amino acid categorical content----- 

def get_Aromaticity(seq):
  return (seq.count('F') + seq.count('H') + seq.count('W') 
          + seq.count('Y'))/len(seq) * 100

# Tiny (A+C+G+S+T)
def get_Tiny(seq):
  return (seq.count('A') + seq.count('C') + seq.count('G') 
          + seq.count('S') + seq.count('T'))/len(seq) * 100

# Small (A+C+D+G+N+P+S+T+V)
def get_Small(seq):
  return (seq.count('A') + seq.count('C') + seq.count('D') 
          + seq.count('G') + seq.count('N') + seq.count('P') 
          + seq.count('S') + seq.count('T') + seq.count('V'))/len(seq) * 100

# Aliphatic (A+I+L+V)
def get_Aliphatic(seq): 
  return (seq.count('A') + seq.count('I') + seq.count('L') 
          + seq.count('V'))/len(seq) * 100

# Nonpolar (A+C+F+G+I+L+M+P+V+W+Y) 
def get_Nonpolar(seq): 
  return (seq.count('A') + seq.count('C') + seq.count('F')
          + seq.count('G') + seq.count('I') + seq.count('L') 
          + seq.count('M') + seq.count('P') + seq.count('V') 
          + seq.count('W') + seq.count('Y')) / len(seq) * 100

# Polar  (D+E+H+K+N+Q+R+S+T)
def get_Polar(seq): 
  return (seq.count('D') + seq.count('E') + seq.count('H')
          + seq.count('K') + seq.count('N') + seq.count('Q') 
          + seq.count('R') + seq.count('S') + seq.count('T'))/ len(seq) * 100

# Basic (H+K+R)
def get_Basic(seq):
  return seq.count('H')+ seq.count('K') + seq.count('R')/len(seq) * 100

# Acidic (D+E)
def get_Acidic(seq): 
  return seq.count('D')+ seq.count('E')/len(seq) * 100

In [77]:
df['Aromatic content'] = df['Seq'].apply(get_Aromaticity)
df['Tiny content'] = df['Seq'].apply(get_Tiny)
df['Small content'] = df['Seq'].apply(get_Small)
df['Aliphatic content'] = df['Seq'].apply(get_Aliphatic)
df['Nonpolar content'] = df['Seq'].apply(get_Nonpolar)
df['Polar content'] = df['Seq'].apply(get_Polar)
df['Basic content'] = df['Seq'].apply(get_Basic)
df['Acidic content'] = df['Seq'].apply(get_Acidic)

In [ ]:
# write out developability data to csv
df.to_csv('results.csv', index=False)